In [1]:
from mitmproxy import http, ctx
import time, re
import logging
import os
from selenium import webdriver
import threading

# import selenium.webdriver.

In [2]:
mitmdump_load_time = 1
script = "mitmdump_script.py"

def StartMITMDump(start_file):
    # TODO change path to MITMdump
    os.system("mitmdump -w " + start_file + " '~t video/webm | ~t video/mp4'")
    
def StopMITMDump(duration, dataPageLoaded = False):
    time.sleep(duration)
    os.system("pkill mitmdump")
    
def ReceiveData(script, file):
    # TODO change path to MITMdump
    os.system("mitmdump -s " + script + " -nr " + file)

def convert_to_bytes_per_second(br):
    return br * 2**10 / 8

In [3]:
# br - kbps
def run_test(br, plr, latency, url, exp_time, headless=True, use_chrome_limiter=True, folder='data/'):
    # Create a network conditions object to limit bandwidth to br bytes per second
    print(f"Network limit: {convert_to_bytes_per_second(br)}")
    to_set = convert_to_bytes_per_second(br)
    network_conditions = {
        'offline': False,
        'latency': latency, # ms
        'download_throughput': to_set, # bytes/s
        'upload_throughput': -1, # bytes/s
        'packet_loss_rate': plr # 0-1
    }
    # Initialize the Chrome browser driver with network conditions and proxy settings
    start_file = folder + f'requests_{br}kbps.txt' #Файл, в который пойдёт весь отфильтрованый поток
    data_file = folder + f'processed_{br}kbps_{latency}ms.txt' #Файл, куда будут сохранены нужные, раскодированные данные
    with open ("save.txt", "w") as f: #Файл для передачи названия конечного файла скрипту
        f.write(data_file)
    os.system(f"rm {data_file}")
    os.system(f"rm {start_file}")
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--proxy-server=localhost:8080')
    chrome_options.add_argument('--ignore-certificate-errors')
    chrome_options.add_argument('--disable-web-security')
    chrome_options.add_argument('--allow-running-insecure-content')
    chrome_options.add_argument('--start-maximized')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-infobars')
    #chrome_options.add_argument('--disable-gpu')
    if headless:
        chrome_options.add_argument('--headless')
    #options.add_argument(url)
    data_page_loaded = False

    print("Loading driver...")
    driver = webdriver.Chrome(options=chrome_options, service_log_path=os.devnull, service_args=['--log-path=./chromedriver.log'])
    if headless:
        # Если запускаем в headless режиме, то нужно увеличить размер окна браузера, иначе не сработает нажатие на кнопку
        driver.set_window_size(1920, 1080)
    thread1 = threading.Thread(target=StopMITMDump, args=(exp_time, data_page_loaded))
    thread2 = threading.Thread(target=StartMITMDump, args=(start_file, ))
    print("Starting mitmdump...")
    thread2.start()
    time.sleep(mitmdump_load_time)
    
    # Navigate to the YouTube video URL
    print("Getting url...")
    driver.get(url)
    print("Setting network conditions...")
    driver.execute_script("document.getElementsByTagName('video')[0].muted = true;")
    if use_chrome_limiter:
        driver.set_network_conditions(**network_conditions)
    
    #Find and click the Play button
    print("Finding play element...")
    play_button = driver.find_element('css selector', '.ytp-play-button')
    print("Clicking...")
    time.sleep(0.1)
    play_button.click()
    
    time.sleep(0.1)
    print("Finding full screen element...")
    full_screen_button = driver.find_element('css selector', '.ytp-fullscreen-button')
    print("Clicking...")
    time.sleep(0.1)
    full_screen_button.click()
    
    #play_video(driver)
    thread1.start()
    # Wait for the video to start playing
    print(f"Sleeping for {exp_time} seconds...")
    time.sleep(exp_time)

    # Stop the video
    action = webdriver.ActionChains(driver)
    action.send_keys(webdriver.Keys.SPACE).perform()

    # Close the browser window
    driver.quit()
    thread1.join()
    thread2.join()
    print('REceiving data...')
    ReceiveData(script, start_file) #Записываем нужные данные в файл при помощи скрипта.
    # Если встречаются неизвестные itag-и, можно найти, чему они соответствуют с помощью youtube-dl -F [ссылка на видео]
    print('Experiment done.')

In [4]:
import pandas as pd
import numpy as np

# br=1000
def get_df(br, latency): #bandwidth rate
    data_file = "data/" + f'processed_{br}kbps_{latency}ms.txt'
    d = []
    itags = {"133":"240 px", "134":"360 px", "135": "480 px", "136": "720 px", "137": "1080 px", "160": "144 px", 
        "264": "1440 px", "298": "720 px", "299": "1080 px", "167": "360 px", "168": "480 px", "169": "720 px",
        "170" : "1080 px", "218": "480 px", "219": "144 px", "242": "240 px", "243": "360 px", "244": "480 px", 
        "245": "480 px", "246": "480 px", "247": "720 px", "248": "1080 px", "271": "1440 px", "278": "144 px",
        '399': '1080 px', '398': '720 px', '397': '480 px', '396': '360 px', '400': '1440 px', '401': '2160 px',
        '395' : '240 px'} 

    names = {
        "itag" : str,
        "mime":  str, 
        "duration": float,  #ms 
        "buffer": float, 
        "clen": float,  # byte
        "request_start": float, 
        "request_complete": float, 
        "response_start": float, 
        "response_complete": float,
        "aitag": str, 
        "range_start": int, 
        "range_end": int, 
    }
    namesList = list(names.keys())
    
    # Если встречаются неизвестные itag-и, можно найти, чему они соответствуют с помощью youtube-dl -F [ссылка на видео]
    with open(data_file, "r") as f:
        a = [line.replace("\n","").split(",") for line in f.readlines()]
    print(f"We made {len(a)} RESPs")
    for idx, val in enumerate(a):
        if val[1] not in itags:
            print(f"BAD {val[1]} ITAG")    
        dataDict = {}
        dataDict["aitag"] = itags[val[1]]
        
        ranges = val[3].split('-')
        dataDict["range_start"] = ranges[0]
        dataDict["range_end"] = ranges[1]
        val.pop(3)
        
        for i in range(9):
            if len(list(filter(lambda s: not s.isnumeric(), val[i].split(".")))) == 0:
                dataDict[namesList[i]] = float(val[i])
            elif val[i] == "None":
                dataDict[namesList[i]] = d[-1][namesList[i]]
            else:
                dataDict[namesList[i]] = val[i]
        
        d.append(dataDict)
        # d.append({"mime":val[0], "itag" : val[1], "aitag":itags[val[1]], "duration":val[2], "range_start":ranges[0], "range_end": ranges[1], 
        #         "buffer":val[4], "clen":val[5],"first_request_byte":val[6], "request_complete": float(val[7]) / 1e6, 
        #         "first_response_byte":val[8], "response_complete": -1 if val[9] == "None" else float(val[9]) / 1e6})
    
    df = pd.DataFrame(d).astype(names)
    return df

df = get_df(1037, 12)
df["request_complete"]  -= df["request_start"][0]
df["response_start"]    -= df["request_start"][0]
df["response_complete"] -= df["request_start"][0]
df["request_start"]     -= df["request_start"][0]

# df["request_complete"]  *= 1000 # ms
# df["response_start"]    *= 1000 # ms
# df["response_complete"] *= 1000 # ms
# df["request_start"]     *= 1000 # ms
df

We made 68 RESPs


,aitag,range_start,range_end,itag,mime,duration,buffer,clen,request_start,request_complete,response_start,response_complete
0,480 px,0,197486,video/mp4,397.0,3342.138,0.0,186965685.0,0.000000,0.000718,0.012607,0.092910
1,720 px,0,8243,video/mp4,398.0,3342.138,0.0,386995805.0,0.058825,0.059753,0.093177,0.095795
2,480 px,197487,213379,video/mp4,397.0,3342.138,9973.0,186965685.0,0.139710,0.140356,0.169979,0.172303
3,720 px,562830,2584285,video/mp4,398.0,3342.138,10677.0,386995805.0,0.181582,0.182082,0.209524,0.746566
4,720 px,2584286,3478122,video/mp4,398.0,3342.138,20375.0,386995805.0,10.964187,10.966025,11.075515,11.280845
...,...,...,...,...,...,...,...,...,...,...,...,...
63,480 px,50721083,52724408,video/mp4,397.0,3342.138,119139.0,186965685.0,782.413472,782.414703,782.486006,783.002324
64,480 px,52724409,54758159,video/mp4,397.0,3342.138,119281.0,186965685.0,814.293137,814.294111,814.321776,814.874453
65,480 px,54758160,56680275,video/mp4,397.0,3342.138,119438.0,186965685.0,846.133383,846.134643,846.176719,846.697549
66,480 px,56680276,58384441,video/mp4,397.0,3342.138,119621.0,186965685.0,883.354352,883.355026,883.368325,883.806559


In [5]:
#lat : eff_rate

def find_effective_rate(latency=0, quality=720, exp_time=60):
    aitag = f"{quality} px"
    l = 1000
    r = 1060
    
    while r - l > 5:
        m = (l + r) // 2
        
        run_test(br=m, plr=0, latency=latency, url="https://www.youtube.com/watch?v=lyh2kAjcmSY", \
                exp_time=exp_time, headless=True, use_chrome_limiter=True, folder='data/')
        df = get_df(m, latency)
        lastAitags = df["aitag"][-6:]
        
        if len(list(filter(lambda s : int(s.split(' ')[0]) >= quality, lastAitags))) > 4:
            r = m
            print(f"upd r={m}")
        else:
            l = m  
            print(f"upd l={m}")
    return r

latency = 12    
# ef_rate = find_effective_rate(latency)
# print(f"effective rate = {ef_rate}, latency = {latency}")


In [32]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

data = {0 : 1031,
        5 : 1000,
        10 : 1018,
        12 : 1037,
        15 : 1048,
        25 : 1056,
        28 : 1112,
        30 : 1093,
        35 : 1093,
        50 : 1132,
        100 : 1120,
        250 : 1156,
        500 : 1192,
        1000 : 1280} 

errors = [ 5,
        5,
        5,
        5,
        5,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20,
        20
        
]

xs = list(data.keys())
ys = list(data.values())
fig = go.Figure()
fig.add_trace(go.Scatter(x =xs, y=ys, mode = "lines+markers"))

fig.update_layout(xaxis_title="Задержка, мс", yaxis_title=r"Эфф. битрейт, кбит/c")
fig.write_image("images/latency.png")
fig.show()

In [51]:
latency = 12
df = get_df(1037, latency)
df["request_complete"]  -= df["request_start"][0]
df["response_start"]    -= df["request_start"][0]
df["response_complete"] -= df["request_start"][0]
df["request_start"]     -= df["request_start"][0]
df["range_start"] /= 1024 / 8
df["range_end"] /= 1024 / 8
df["clen"] /= 1024 / 8
df["buffer"] /= 1024 / 8

W = 6 # window width
rates = []
times = []
N = len(df)
for i in range(N - W + 1):
    recieved = 0
    for k in range(W):
        id = i + k
        recieved += df["range_end"][id] - df["range_start"][id]
    timeLeft = df["response_complete"][i + W - 1] - df["request_start"][i]
    
    rates.append(recieved / timeLeft)
    times.append(df["request_start"][i + W // 2 - 1])
    
rates = rates[2:]
times = times[2:]




fig = go.Figure()
fig.add_trace(go.Scatter(x = df["request_start"], y = df["buffer"], name = "Буфер", mode = "lines+markers"))

netRate = (df["range_end"] - df["range_start"]) / (df["response_complete"] - df["request_start"])
streamBitRate = df["clen"] / df["duration"]
fig.add_trace(go.Scatter(x = times, y = rates, mode = "lines+markers", name=f"Средняя пропускная способность в окне {W}"))
# fig.add_trace(go.Scatter(x = df["request_start"], y = netRate, name = "Пропускная способность сети", mode = "lines+markers"))
fig.add_trace(go.Scatter(x = df["request_start"], y = streamBitRate, name = "Битрейт видео кБ/с", mode = "lines+markers"))
fig.add_trace(go.Scatter(x = [0, 880], y = [1037] * 2, name = "Ограничение битрейта", mode = "lines+markers"))

fig.update_layout(xaxis_title="Время, с", yaxis_title="Килобайт или килобайт/с", title=f"Задержка {latency} мс")
fig.write_image(f"images/time{latency}.png")
fig.show()


We made 68 RESPs


In [52]:
df

,aitag,range_start,range_end,itag,mime,duration,buffer,clen,request_start,request_complete,response_start,response_complete
0,480 px,0.000000,1542.859375,video/mp4,397.0,3342.138,0.000000,1.460669e+06,0.000000,0.000718,0.012607,0.092910
1,720 px,0.000000,64.398438,video/mp4,398.0,3342.138,0.000000,3.023405e+06,0.058825,0.059753,0.093177,0.095795
2,480 px,1542.867188,1667.023438,video/mp4,397.0,3342.138,77.914062,1.460669e+06,0.139710,0.140356,0.169979,0.172303
3,720 px,4397.109375,20189.726562,video/mp4,398.0,3342.138,83.414062,3.023405e+06,0.181582,0.182082,0.209524,0.746566
4,720 px,20189.734375,27172.828125,video/mp4,398.0,3342.138,159.179688,3.023405e+06,10.964187,10.966025,11.075515,11.280845
...,...,...,...,...,...,...,...,...,...,...,...,...
63,480 px,396258.460938,411909.437500,video/mp4,397.0,3342.138,930.773438,1.460669e+06,782.413472,782.414703,782.486006,783.002324
64,480 px,411909.445312,427798.117188,video/mp4,397.0,3342.138,931.882812,1.460669e+06,814.293137,814.294111,814.321776,814.874453
65,480 px,427798.125000,442814.648438,video/mp4,397.0,3342.138,933.109375,1.460669e+06,846.133383,846.134643,846.176719,846.697549
66,480 px,442814.656250,456128.445312,video/mp4,397.0,3342.138,934.539062,1.460669e+06,883.354352,883.355026,883.368325,883.806559


Пропускная способность в окне в 6 сегментов

In [45]:
W = 6 # window width
rates = []
times = []
N = len(df)
for i in range(N - W + 1):
    recieved = 0
    for k in range(W):
        id = i + k
        recieved += df["range_end"][id] - df["range_start"][id]
    timeLeft = df["response_complete"][i + W - 1] - df["request_start"][i]
    
    rates.append(recieved / timeLeft)
    times.append(df["request_start"][i + W // 2 - 1])
    
    fig = go.Figure()
fig.add_trace(go.Scatter(x = times, y = rates, mode = "lines+markers"))

fig.update_layout(xaxis_title="Время, с", yaxis_title="килобайт/с", title=f"усредненная пропускная способность, окно в 6 сегментов")
fig.write_image(f"images/rate.png")
fig.show()


In [37]:
latency=500
br=1192
run_test(br=br, plr=0, latency=latency, url="https://www.youtube.com/watch?v=lyh2kAjcmSY", \
                exp_time=500, headless=True, use_chrome_limiter=True, folder='data/')

Network limit: 152576.0
Loading driver...


rm: cannot remove 'data/processed_1192kbps_500ms.txt': No such file or directory


Starting mitmdump...
[16:13:44.492] Only processing flows that match "~t video/webm | ~t video/mp4"
[16:13:44.496] HTTP(S) proxy listening at *:8080.
Getting url...
[16:13:45.183][[::1]:49250] client connect
[16:13:50.382][[::1]:49250] server connect www.youtube.com:443 (209.85.233.198:443)
[16:13:51.115][[::1]:52754] client connect
[16:13:51.240][[::1]:52754] server connect fonts.googleapis.com:443 (142.250.74.170:443)
[16:13:51.412][[::1]:52760] client connect
[16:13:51.414][[::1]:52768] client connect
[16:13:51.415][[::1]:52770] client connect
[16:13:51.417][[::1]:52772] client connect
[16:13:51.429][[::1]:52778] client connect
[16:13:51.514][[::1]:52768] server connect rr4---sn-4pvgq-n8v6.googlevideo.com:443 (80.77.175.47:443)
[16:13:51.515][[::1]:52770] server connect rr4---sn-4pvgq-n8v6.googlevideo.com:443 (80.77.175.47:443)
[16:13:51.539][[::1]:52772] server connect i.ytimg.com:443 (64.233.164.119:443)
[16:13:51.539][[::1]:52760] server connect i.ytimg.com:443 (64.233.164.119:44

In [38]:

df = get_df(br, latency)
df["request_complete"]  -= df["request_start"][0]
df["response_start"]    -= df["request_start"][0]
df["response_complete"] -= df["request_start"][0]
df["request_start"]     -= df["request_start"][0]
df["range_start"] /= 1024 / 8
df["range_end"] /= 1024 / 8
df["clen"] /= 1024 / 8
df["buffer"] /= 1024 / 8

fig = go.Figure()
fig.add_trace(go.Scatter(x = df["request_start"], y = df["buffer"], name = "Буфер", mode = "lines+markers"))

netRate = (df["range_end"] - df["range_start"]) / (df["response_complete"] - df["request_start"])
streamBitRate = df["clen"] / df["duration"]
# fig.add_trace(go.Scatter(x = df["request_start"], y = netRate, name = "Пропускная способность сети", mode = "lines+markers"))
fig.add_trace(go.Scatter(x = df["request_start"], y = streamBitRate, name = "Битрейт видео кБ/с", mode = "lines+markers"))
fig.add_trace(go.Scatter(x = [0, df["request_start"].values[-1]], y = [br] * 2, name = "Ограничение битрейта", mode = "lines+markers"))

fig.update_layout(xaxis_title="Время, с", yaxis_title="Килобайт или килобайт/с", title=f"Задержка {latency} мс")
fig.write_image(f"images/time{latency}.png")
fig.show()

We made 95 RESPs
